# Cardinality constrained portfolio optimization

### Import libraries

In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

(CVXPY) Jun 04 01:49:36 AM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


SCIP solver can be installed with `conda install -c conda-forge pyscipopt` or `pip install pyscipopt`.

In [2]:
from pyscipopt.scip import Model

ModuleNotFoundError: No module named 'pyscipopt'

In [ ]:
np.random.seed(940932)

In [ ]:
print(cp.installed_solvers())

### Generate random data for 10 stocks

In [ ]:
# Random data for 10 stocks
n = 10
Q = np.random.random((n,n))
Q = np.dot(Q,Q.T)/1000
# Q
mu = np.random.rand(n) / 100
# mu

### Cardinality constrained mean-variance optimization problem

\begin{equation}
 \begin{array}{rll}
  \displaystyle \min_{w,z} & w^T Q w \\
  \mbox{s.t.} & \sum_{i=1}^n w_i = 1 \\
  & \sum_{i=1}^n z_i = K \\
  & l \cdot z \leq w \leq u \cdot z \\
  & z = \{0,1\}
 \end{array}
\end{equation}

### Define and solve model using CVXPY modeling environment

Compute minimum variance portfolio with cardinality $K=2$

In [ ]:
w = cp.Variable(n)
z = cp.Variable(n, boolean=True)
K = cp.Parameter()
K.value = 2
prob = cp.Problem(cp.Minimize(cp.quad_form(w, Q)),
                 [sum(w) == 1,
                  sum(z) == K,
                  w - z <= 0,
                  w >= 0])
prob.solve(solver='SCIP', verbose=True)

print("\nSolution status: ", prob.status)
print("Solution optimal value: ", prob.value)
print("Solution w: ")
print(w.value)

In [ ]:
w_minVar = w.value
var_minVar = np.dot(w_minVar, np.dot(Q, w_minVar))
print("Minimum variance portfolio with cardinality K=2:\n")
print("Solution status =", prob.status)
print("Solution value =", prob.value)
print("Variance       =", var_minVar)
print("Standard deviation =", np.sqrt(var_minVar))

Compute minimum variance portfolios with cardinality $K=1,2,...,10$

In [ ]:
w_cardP = []
var_cardP = []
p_cardP = []
for card in range(1,n+1):
    K.value = card
    p_cardP.append(card)
    prob.solve(verbose=False)
    w_cardP.append(w.value)
    var_cardP.append(np.dot(w.value, np.dot(Q, w.value)))

In [ ]:
fig=plt.figure(figsize=(8,4), dpi= 100, facecolor='w', edgecolor='k')
ax = plt.gca()
ax.plot(p_cardP, np.sqrt(var_cardP), 'bo')

ax.set_xlim([0,n+1])
ax.set_xlabel('Portfolio cardinality')
ax.set_ylabel('Portfolio standard deviation')
plt.title('Minimum Variance Portfolios with Cardinality Constraint')
ax.grid()
plt.show()

In [ ]:
w_cardP